In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('/data', exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Buisness', 'Sci/Tech']

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
list(train_dataset[0])

In [ ]:
for x in train_dataset[:5]:
    print(f"**{classes[x[0]]}** -> {x[1]}")

In [ ]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)